In [21]:
from re import S
import torch
import numpy as np
import pandas as pd
import json
import pickle
# from torch._C import R
import tqdm
from torch.random import seed
from utils.functions import sentiment_data_filtering, get_user_item_dict, get_feature_list, \
    get_user_attention_matrix, get_item_quality_matrix, sample_training_pairs
import pandas as pd
import argparse
from utils.argument_amazon import arg_parse_train_base, arg_parser_preprocessing
import ast
from sys import exit
import torch
import pickle
import os
from pathlib import Path
from utils.argument_amazon import arg_parse_exp_optimize
from models.models import BaseRecModel, ExpOptimizationModel

In [26]:
import pickle

# Path to your pickle file
pickle_file_path = "Datasets/reviews_a2.pickle"

# Open the pickle file in binary mode for reading
with open(pickle_file_path, "rb") as file:
    # Load the object from the pickle file
    loaded_object = pickle.load(file)


In [28]:
df

,user,unixReviewTime,item,text,sentence
0,AJ6B83I4YJHYW,1372636800,0000031887,This was a gift for a two year old and a five ...,NaN
1,A1R377IPZOKLMM,1284768000,1608299953,I really enjoy the Rosetta Stone software; it'...,NaN
2,A122XGIAS5WN57,1379894400,B00006XXGO,"Nice fit, looks like the picture. Delivered pr...",NaN
3,A3KR8RNNPF2XY,1403222400,B00006XXGO,"I now have several in white, red, and black......",NaN
4,A2TY9KJ0B5LI0P,1355270400,B000072UMJ,These run bigger than normal tennis shoes...my...,NaN
...,...,...,...,...,...
10249,A34BZM6S9L7QI4,1400889600,B00JZSDW8A,These are as pretty in person as they are on h...,NaN
10250,A38VCIVAE3OXD3,1405987200,B00K0BPFY0,Lots of compliments on this shirt. At 5'3&#34...,NaN
10251,A34BZM6S9L7QI4,1403308800,B00K5T4NHC,I slipped this in among my son's other moistur...,"[(size, true, true to size and a great value, 1)]"
10252,A1M1Y5UGONAW06,1405641600,B00K8J06CK,"I'm a huge fan of Azules, and wear an Azules ...",NaN


In [4]:
t_args = arg_parse_train_base()
exp_args = arg_parse_exp_optimize()
ap = arg_parser_preprocessing()

In [27]:
df = pd.DataFrame(loaded_object)
print(df.head(5))

# Custom function to replace NaN with empty list
df.isna().sum()

             user  unixReviewTime        item  \
0   AJ6B83I4YJHYW      1372636800  0000031887   
1  A1R377IPZOKLMM      1284768000  1608299953   
2  A122XGIAS5WN57      1379894400  B00006XXGO   
3   A3KR8RNNPF2XY      1403222400  B00006XXGO   
4  A2TY9KJ0B5LI0P      1355270400  B000072UMJ   

                                                text sentence  
0  This was a gift for a two year old and a five ...      NaN  
1  I really enjoy the Rosetta Stone software; it'...      NaN  
2  Nice fit, looks like the picture. Delivered pr...      NaN  
3  I now have several in white, red, and black......      NaN  
4  These run bigger than normal tennis shoes...my...      NaN  


user                 0
unixReviewTime       0
item                 0
text                 0
sentence          7043
dtype: int64

In [9]:
df

,user,unixReviewTime,item,text,sentence
0,ATX7CZYFXI1KW,1365552000,B00A8ZEG0C,Awesome - lots of bling and holds in phone nic...,"[(phone, nice, Awesome - lots of bling and hol..."
1,A2R99I2RXNYT8A,1375142400,B00A8ZEG0C,I really lie this dust plug it fits well in my...,NaN
2,ATX7CZYFXI1KW,1359849600,B006RBZ6LE,Perfect color. Nice addition to the I-phone f...,NaN
3,ATX7CZYFXI1KW,1359849600,B005KDRA5K,Nice case - all in the bling that I was lookin...,"[(protector, hard, Apple iPhone 4 and 4S snap ..."
4,A36Y6BFEEGBP5X,1363910400,B005KDRA5K,This case protects the front and back and look...,"[(looks, good, This case protects the front an..."
...,...,...,...,...,...
2998,A1DGB6HY5C7LXU,1398902400,B00JMO1V3A,I love this cover by Fosmon. It's easy to ins...,"[(product, great, Thanks for the great product..."
2999,A382P8ZWX1JDP1,1398297600,B00JMO1V3A,The built in screen is pretty awesome. I love ...,NaN
3000,A1Q0J72AMYGTNT,1400198400,B00JMO1V3A,This case was easy to put on my phone and has ...,NaN
3001,A382P8ZWX1JDP1,1398556800,B00J340RVG,Really great quality for the cost. Works just ...,NaN


In [8]:
df = df.fillna(0)

In [20]:
print("Review:", df[df['sentence']!=0]['text'][0])
print("Result:",df[df['sentence']!=0]['sentence'][0])

Review: awesome! stays on, and looks great. can be used on multiple apple products.  especially having nails, it helps to have an elevated key.
Result: [('looks', 'great', 'and looks great', 1)]


In [11]:
df.isna().sum()

user              0
unixReviewTime    0
item              0
text              0
sentence          0
dtype: int64

In [12]:
from re import S
import torch
import numpy as np
import json
import pickle
# from torch._C import R
import tqdm
from torch.random import seed
from utils.functions import sentiment_data_filtering, get_user_item_dict, get_feature_list, \
    get_user_attention_matrix, get_item_quality_matrix, sample_training_pairs


class AmazonDataset():
    def __init__(self, preprocessing_args):
        super().__init__()
        self.args = preprocessing_args
        self.sentiment_data = None  # [userID, itemID, [fos triplet 1], [fos triplet 2], ...]

        self.user_name_dict = {}  # rename users to integer names
        self.item_name_dict = {}
        self.feature_name_dict = {}

        self.features = []  # feature list
        self.users = []
        self.items = []

        # the interacted items for each user, sorted with date {user:[i1, i2, i3, ...], user:[i1, i2, i3, ...]}
        self.user_hist_inter_dict = {}
        # the interacted users for each item
        self.item_hist_inter_dict = {}  

        self.user_num = None
        self.item_num = None
        self.feature_num = None  # number of features

        self.user_feature_matrix = None  # user aspect attention matrix
        self.item_feature_matrix = None  # item aspect quality matrix

        self.training_data = None
        self.test_data = None
        self.pre_processing()
        self.get_user_item_feature_matrix()
        self.sample_training()  # sample training data, for traning BPR loss
        self.sample_test()  # sample test data

    def pre_processing(self,):
        sentiment_data = []
        for index, row in df.iterrows():
            user = row['user']
            item = row['item']
            sentiment_data.append([user, item])
            fosr_data = row['sentence']
            if fosr_data == 0:
                fosr_data = []
                continue
            for all in fosr_data:
                sentiment = all[3]
                if sentiment == 1:
                    sentiment = "+1"
                elif sentiment == -1:
                    sentiment = "-1"
                if sentiment != "+1" and sentiment != "-1":
                    continue
                sentiment_data[-1].append([all[0], all[1], sentiment])
#                 print(sentiment_data)
        # Appending data to sentiment_data
        sentiment_data = np.array(sentiment_data)
        sentiment_data = sentiment_data_filtering(
            sentiment_data, 
            self.args.user_thresh, 
            self.args.feature_thresh)
        user_dict, item_dict = get_user_item_dict(sentiment_data)  # not sorted with time
        user_item_date_dict = {}   # {(user, item): date, (user, item): date ...}  # used to remove duplicate

        for index, record in df.iterrows():
            user = record['user']
            item = record['item']
            date = record['unixReviewTime']
            if user in user_dict and item in user_dict[user] and (user, item) not in user_item_date_dict:
                user_item_date_dict[(user, item)] = date

        # remove the (user, item) not exist in the official dataset, possibly due to update?
        sentiment_data = [row for row in sentiment_data if (row[0], row[1]) in user_item_date_dict]
        sentiment_data = sentiment_data_filtering(sentiment_data, self.args.user_thresh, self.args.feature_thresh)
        user_dict, item_dict = get_user_item_dict(sentiment_data)
        for key in list(user_item_date_dict.keys()):
            if key[0] not in user_dict or key[1] not in user_dict[key[0]]:
                del user_item_date_dict[key]
        
        # rename users, items, and features to integer names
        user_name_dict = {}
        item_name_dict = {}
        feature_name_dict = {}
        features = get_feature_list(sentiment_data)
        
        count = 0
        for user in user_dict:
            if user not in user_name_dict:
                user_name_dict[user] = count
                count += 1
        count = 0
        for item in item_dict:
            if item not in item_name_dict:
                item_name_dict[item] = count
                count += 1
        count = 0
        for feature in features:
            if feature not in feature_name_dict:
                feature_name_dict[feature] = count
                count += 1
        
        for i in range(len(sentiment_data)):
            sentiment_data[i][0] = user_name_dict[sentiment_data[i][0]]
            sentiment_data[i][1] = item_name_dict[sentiment_data[i][1]]
            for j in range(len(sentiment_data[i]) - 2):
                sentiment_data[i][j+2][0] = feature_name_dict[sentiment_data[i][j + 2][0]]

        renamed_user_item_date_dict = {}
        for key, value in user_item_date_dict.items():
            renamed_user_item_date_dict[user_name_dict[key[0]], item_name_dict[key[1]]] = value
        user_item_date_dict = renamed_user_item_date_dict

        # sort with date
        user_item_date_dict = dict(sorted(user_item_date_dict.items(), key=lambda item: item[1]))

        user_hist_inter_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}, sort with time
        item_hist_inter_dict = {}
        # ranked_user_item_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}
        for key, value in user_item_date_dict.items():
            user = key[0]
            item = key[1]
            if user not in user_hist_inter_dict:
                user_hist_inter_dict[user] = [item]
            else:
                user_hist_inter_dict[user].append(item)
            if item not in item_hist_inter_dict:
                item_hist_inter_dict[item] = [user]
            else:
                item_hist_inter_dict[item].append(user)

        user_hist_inter_dict = dict(sorted(user_hist_inter_dict.items()))
        item_hist_inter_dict = dict(sorted(item_hist_inter_dict.items()))

        users = list(user_hist_inter_dict.keys())
        items = list(item_hist_inter_dict.keys())

        self.sentiment_data = sentiment_data
        self.user_name_dict = user_name_dict
        self.item_name_dict = item_name_dict
        self.feature_name_dict = feature_name_dict
        self.user_hist_inter_dict = user_hist_inter_dict
        self.item_hist_inter_dict = item_hist_inter_dict
        self.users = users
        self.items = items
        self.features = features
        self.user_num = len(users)
        self.item_num = len(items)
        self.feature_num = len(features)
        return True
    
    def get_user_item_feature_matrix(self,):
        # exclude test data from the sentiment data to construct matrix
        train_u_i_set = set()
        for user, items in self.user_hist_inter_dict.items():
            items = items[:-self.args.test_length]
            for item in items:
                train_u_i_set.add((user, item))

        train_sentiment_data = []
        for row in self.sentiment_data:
            user = row[0]
            item = row[1]
            if (user, item) in train_u_i_set:
                train_sentiment_data.append(row)
        self.user_feature_matrix = get_user_attention_matrix(
            train_sentiment_data, 
            self.user_num, 
            self.features, 
            max_range=5)
        self.item_feature_matrix = get_item_quality_matrix(
            train_sentiment_data, 
            self.item_num, 
            self.features, 
            max_range=5)
        return True
    
    def sample_training(self):
        print('======================= sample training data =======================')
        # print(self.user_feature_matrix.shape, self.item_feature_matrix.shape)
        training_data = []
        item_set = set(self.items)
        for user, items in self.user_hist_inter_dict.items():
            items = items[:-(self.args.test_length+self.args.val_length)]
            training_pairs = sample_training_pairs(
                user, 
                items, 
                item_set, 
                self.args.sample_ratio)
            for pair in training_pairs:
                training_data.append(pair)
        print('# training samples :', len(training_data))
        self.training_data = np.array(training_data)
        return True
    
    def sample_test(self):
        print('======================= sample test data =======================')
        user_item_label_list = []  # [[u, [item1, item2, ...], [l1, l2, ...]], ...]
        for user, items in self.user_hist_inter_dict.items():
            items = items[-(self.args.test_length+self.args.val_length):]
            user_item_label_list.append([user, items, np.ones(len(items))])  # add the test items
            negative_items = [item for item in self.items if 
                item not in self.user_hist_inter_dict[user]]  # the not interacted items
            negative_items = np.random.choice(np.array(negative_items), self.args.neg_length, replace=False)
            user_item_label_list[-1][1] = np.concatenate((user_item_label_list[-1][1], negative_items), axis=0)
            user_item_label_list[-1][2] = np.concatenate((user_item_label_list[-1][2], np.zeros(self.args.neg_length)), axis=0)
        print('# test samples :', len(user_item_label_list))
        self.test_data = np.array(user_item_label_list)
        return True

    def save(self, save_path):
        return True
    
    def load(self):
        return False


def amazon_preprocessing(pre_processing_args):
    rec_dataset = AmazonDataset(pre_processing_args)
    return rec_dataset

In [13]:
amazon_preprocessing(ap)

/tmp/ipykernel_6505/3425732973.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)


======================= filtering sentiment data =======================
original review length:  3003
original user length:  254
original item length:  2062
original feature length:  32
valid review length:  472
valid user:  203
valid item :  421
valid feature length:  12
user dense is: 2.3251231527093594
======================= filtering sentiment data =======================
original review length:  472
original user length:  203
original item length:  421
original feature length:  12
valid review length:  472
valid user:  203
valid item :  421
valid feature length:  12
user dense is: 2.3251231527093594
======================= sample training data =======================
# training samples : 538
======================= sample test data =======================
# test samples : 203


/home/amparikh/Untitled Folder/utils/functions.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)
/home/amparikh/Untitled Folder/utils/functions.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)
/tmp/ipykernel_6505/3425732973.py:236: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=ob

In [14]:
import torch
import numpy as np
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader
from utils.argument_amazon import arg_parse_train_base, arg_parser_preprocessing
from models.data_loaders import UserItemInterDataset
from models.models import BaseRecModel
from utils.evaluate_functions import compute_ndcg

def train_base_recommendation(train_args, pre_processing_args):
    if exp_args.gpu:
        device = torch.device('cuda:%s' % exp_args.cuda)
    else:
        device = 'cpu'
    rec_dataset = amazon_preprocessing(pre_processing_args)
    Path(pre_processing_args.save_path).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(pre_processing_args.save_path, pre_processing_args.dataset + "_dataset_obj.pickle"), 'wb') as outp:
        pickle.dump(rec_dataset, outp, pickle.HIGHEST_PROTOCOL)

    train_loader = DataLoader(dataset=UserItemInterDataset(rec_dataset.training_data, 
                                rec_dataset.user_feature_matrix, 
                                rec_dataset.item_feature_matrix),
                          batch_size=train_args.batch_size,
                          shuffle=True)

    model = BaseRecModel(rec_dataset.feature_num).to(device)
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=train_args.lr, weight_decay=train_args.weight_decay)

    out_path = os.path.join("./logs", train_args.dataset + "_logs")
    Path(out_path).mkdir(parents=True, exist_ok=True)

    ndcg = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            train_args.rec_k, 
            model, 
            device)
    print('init ndcg:', ndcg)
    for epoch in tqdm.trange(train_args.epoch):
        model.train()
        optimizer.zero_grad()
        losses = []
        for user_behaviour_feature, item_aspect_feature, label in train_loader:
            user_behaviour_feature = user_behaviour_feature.to(device)
            item_aspect_feature = item_aspect_feature.to(device)
            label = label.float().to(device)
            out = model(user_behaviour_feature, item_aspect_feature).squeeze()
            loss = loss_fn(out, label)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.to('cpu').detach().numpy())
            ave_train = np.mean(np.array(losses))
        print('epoch %d: ' % epoch, 'training loss: ', ave_train)
        # compute necg
        if epoch % 10 == 0:
            ndcg = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            train_args.rec_k, 
            model, 
            device)
            print('epoch %d: ' % epoch, 'training loss: ', ave_train, 'NDCG: ', ndcg)
    torch.save(model.state_dict(), os.path.join(out_path, "model.model"))
    return 0



In [15]:
#   # training arguments
if t_args.gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = t_args.cuda
    print("Using CUDA", t_args.cuda)
else:
    print("Using CPU")
train_base_recommendation(t_args, ap)

Using CUDA 0


/tmp/ipykernel_6505/3425732973.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)


======================= filtering sentiment data =======================
original review length:  3003
original user length:  254
original item length:  2062
original feature length:  32
valid review length:  472
valid user:  203
valid item :  421
valid feature length:  12
user dense is: 2.3251231527093594
======================= filtering sentiment data =======================
original review length:  472
original user length:  203
original item length:  421
original feature length:  12
valid review length:  472
valid user:  203
valid item :  421
valid feature length:  12
user dense is: 2.3251231527093594
======================= sample training data =======================
# training samples : 538
======================= sample test data =======================
# test samples : 203


/home/amparikh/Untitled Folder/utils/functions.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)
/home/amparikh/Untitled Folder/utils/functions.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentiment_data = np.array(sentiment_data)
/tmp/ipykernel_6505/3425732973.py:236: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=ob

init ndcg: 0.008698861623275326


...

100%|█████████▉| 498/500 [00:26<00:00, 22.81it/s]

epoch 490:  training loss:  0.12641688 NDCG:  0.002149739044472232
epoch 491:  training loss:  0.12242709
epoch 492:  training loss:  0.12464522
epoch 493:  training loss:  0.12975758
epoch 494:  training loss:  0.13427877
epoch 495:  training loss:  0.1166538
epoch 496:  training loss:  0.118282676
epoch 497:  training loss:  0.11565407
epoch 498:  training loss:  0.1175118


100%|██████████| 500/500 [00:26<00:00, 19.03it/s]

epoch 499:  training loss:  0.119153365


0

In [16]:

# if exp_args.gpu:
#     device = torch.device('cuda:%s' % exp_args.cuda)
# else:
#     device = 'cpu'
# print(device)

In [17]:



def generate_explanation(exp_args):
    if exp_args.gpu:
        device = torch.device('cuda:%s' % exp_args.cuda)
    else:
        device = 'cpu'
    # import dataset
    with open(os.path.join(exp_args.data_obj_path, exp_args.dataset + "_dataset_obj.pickle"), 'rb') as inp:
        rec_dataset = pickle.load(inp)
    base_model = BaseRecModel(rec_dataset.feature_num).to(device)
    base_model.load_state_dict(torch.load(os.path.join(exp_args.base_model_path, exp_args.dataset+"_logs", "model.model")))
    base_model.eval()
    #  fix the rec model
    for param in base_model.parameters():
        param.requires_grad = False
    
    # Create optimization model
    opt_model = ExpOptimizationModel(
        base_model=base_model,
        rec_dataset=rec_dataset,
        device = device,
        exp_args=exp_args,
    )
    

    opt_model.generate_explanation()
    opt_model.user_side_evaluation()
    opt_model.model_side_evaluation()
    Path(exp_args.save_path).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(exp_args.save_path, exp_args.dataset + "_explanation_obj.pickle"), 'wb') as outp:
        pickle.dump(opt_model, outp, pickle.HIGHEST_PROTOCOL)
    return True


if __name__ == "__main__":
    exp_args = arg_parse_exp_optimize()
    generate_explanation(exp_args)

100%|██████████| 203/203 [32:37<00:00,  9.64s/it]


ave num:  1.0199081163859112 ave complexity:  2.390573562962926
user's perspective:
ave pre:  1.0   ave rec:  1.0   ave f1:  1.0
model's perspective:
ave PN:  0.9709035222052067   ave PS:  0.5620214395099541   ave F_{NS}:  0.7119312540445771
